Projet 


In [5]:
#Importation Package
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [7]:
#Importation des données 
df_complement_individu_2016 = pd.read_csv('R_COMPLEMENT_INDIVIDU_2016.csv',encoding='ISO-8859-1',delimiter=';')
df_individu_2016 = pd.read_csv('R_INDIVIDU_2016.csv',encoding='ISO-8859-1',delimiter=';',dtype = {'DATE_NAISS_A':'str','DATE_NAISS_M':'str','DATE_NAISS_J':'str'})
df_magasin = pd.read_csv('R_MAGASIN.csv',encoding='ISO-8859-1',delimiter=';')
df_referentiel = pd.read_csv('R_REFERENTIEL.csv',encoding='ISO-8859-1',delimiter=';',dtype = {'EAN':'str'})
df_tickets_2016 = pd.read_csv('R_TICKETS_2016.csv',encoding='ISO-8859-1',delimiter=';',dtype = {'EAN':'str'})
df_typo_produit = pd.read_csv('R_TYPO_PRODUIT.csv',encoding='ISO-8859-1',delimiter=';')

On attend  autant d'individu que dans la table Individu, on enlevera les individus n'ayant pas de complement en filtrant sur les données manquante et on fera une jointure sur l'ID 

In [8]:
individu = df_individu_2016.merge(df_complement_individu_2016, on='ID_INDIVIDU', how='left')
individu.drop(["PROFESSION"], axis=1, inplace=True)
#individu.dropna(inplace=True)
individu

,ID_INDIVIDU,ID_FOYER,CIVILITE,SEXE,CATEGORIE_PROF,DATE_NAISS_A,DATE_NAISS_M,DATE_NAISS_J,DATE_CREATION_CARTE,CODE_MAGASIN,PAYS,ETAT,TAILLE,TAILLE_SG,TAILLE_BONNET
0,4,175898,0,0,NaN,NaN,NaN,NaN,01/01/1989,751,FR,N,38.0,95.0,B
1,27,0,3,2,3.0,1967,9,1,08/08/2007,942,FR,N,38.0,85.0,B
2,34,127060,3,2,6.0,1953,3,4,27/07/2007,942,FR,N,42.0,95.0,B
3,37,0,2,2,3.0,1964,8,21,11/07/2007,942,FR,N,42.0,90.0,B
4,47,0,3,2,NaN,1972,3,31,23/04/2008,941,FR,N,40.0,90.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36152,241454,237533,3,2,NaN,2000,1,1,12/09/2016,7511,FR,C,NaN,NaN,NaN
36153,241459,237577,2,2,8.0,1991,3,19,12/09/2016,661,FR,C,1.0,85.0,b
36154,241460,237576,3,2,NaN,1955,8,3,12/09/2016,661,FR,C,2.0,85.0,b
36155,241473,237561,3,2,NaN,1978,5,25,12/09/2016,381,FR,C,NaN,NaN,NaN


In [9]:
date_extract = datetime(2016, 8, 31)

individu["DATE_NAISS"] = individu["DATE_NAISS_A"] + "-" + individu["DATE_NAISS_M"] + "-" + individu["DATE_NAISS_J"]
individu["AGE"] =  date_extract - pd.to_datetime(individu["DATE_NAISS"], format='%Y-%m-%d',errors='coerce') 

individu["DATE_CREATION_CARTE"] =  pd.to_datetime(individu["DATE_CREATION_CARTE"], format='%d/%m/%Y',errors='coerce')
individu["ANCIENNETE"] = date_extract - individu["DATE_CREATION_CARTE"]

individu["AGE"] = individu["AGE"].astype('timedelta64[Y]')
individu["ANCIENNETE"] = individu["ANCIENNETE"].astype('timedelta64[M]')

In [10]:
individu.loc[(individu["AGE"] < 15) | (individu["AGE"] > 80), "AGE"] = np.nan
print(individu["AGE"].max(), individu["AGE"].min())

individu.loc[(individu["ANCIENNETE"] < 0), "ANCIENNETE"] = np.nan
print(individu["ANCIENNETE"].max(), individu["ANCIENNETE"].min())


80.0 15.0
331.0 0.0


In [11]:
DF_INDIVIDU = individu

In [12]:
df_ref_produit = df_referentiel.merge(df_typo_produit, on='MODELE', how='left')
df_ticket_ref = df_tickets_2016.merge(df_ref_produit, on='EAN', how='left')
df_matrice_travail = df_ticket_ref.merge(df_magasin, on='CODE_BOUTIQUE', how='left')

df_matrice_travail["DATE_ACHAT"] = pd.to_datetime(df_matrice_travail["DATE_ACHAT"], format='%d/%m/%Y',errors='coerce')
df_matrice_travail = df_matrice_travail.loc[(df_matrice_travail["DATE_ACHAT"] > '2014-08-31')]

In [13]:
selected_columns = ['DATE_ACHAT','EAN', 'ID_INDIVIDU', 'ID_FOYER', 'CODE_LIGNE', 'TYPE_LIGNE', 'NUM_TICKET', 'QUANTITE', 'PRIX_AP_REMISE', 'REMISE', 'REMISE_VALEUR', 'CODE_BOUTIQUE', 'REGIONS', 'CENTRE_VILLE', 'TYPE_MAGASIN', 'REGIONS_COMMERCIAL', 'MODELE', 'Ligne', 'Famille']
df_matrice_travail = df_matrice_travail[selected_columns]



In [14]:

df_matrice_travail.loc[df_matrice_travail["REMISE_VALEUR"] > 100, "REMISE_VALEUR"] = np.nan
df_matrice_travail['PRIX_OK'] = df_matrice_travail.apply(lambda x :0 if x['MODELE'] in ['FAVO','FAVORI'] else x['PRIX_AP_REMISE'], axis=1)

df_matrice_travail.loc[df_matrice_travail['MODELE'].isin(['FAVO','FAVORI'])]['PRIX_OK'].describe()
df_matrice_travail.loc[df_matrice_travail['MODELE'].isin(['FAVO','FAVORI'])==False]['PRIX_OK'].describe()

df_matrice_travail['CENTRE_VILLE'].value_counts()
df_matrice_travail['CENTRE_VILLE'] = df_matrice_travail.apply(lambda x : 'Centre ville'
                    if x['CENTRE_VILLE'] in ['Centre ville']
                    else 'Centre commercial'
                    if  x['CENTRE_VILLE'] in ['Centre Co','Centre Commercial']
                                                              else 'NC', axis=1)

Matrice_travail_OK = df_matrice_travail

In [16]:
Ticket_174591 = Matrice_travail_OK.loc[Matrice_travail_OK["ID_INDIVIDU"] == 174591]

In [17]:
Matrice_travail_OK = Matrice_travail_OK.loc[Matrice_travail_OK["DATE_ACHAT"] > '2014-08-31']
Matrice_travail_OK 

,DATE_ACHAT,EAN,ID_INDIVIDU,ID_FOYER,CODE_LIGNE,TYPE_LIGNE,NUM_TICKET,QUANTITE,PRIX_AP_REMISE,REMISE,REMISE_VALEUR,CODE_BOUTIQUE,REGIONS,CENTRE_VILLE,TYPE_MAGASIN,REGIONS_COMMERCIAL,MODELE,Ligne,Famille,PRIX_OK
1,2015-12-14,3585211731150,4,175898,81,SALE,10,1,69,0,0.0,730,Province,Centre ville,Affilié,Province,222,Homewear,Homewear_Ensemble,69
4,2016-07-02,3585211668319,4,175898,81,SALE,513,1,75,0,0.0,920,Paris,Centre commercial,Succursale,Paris,503,Bain,Bain_Maillot,75
5,2016-06-21,3585211799914,4,175898,81,SALE,545,1,12,3,20.0,920,Paris,Centre commercial,Succursale,Paris,124,Corseterie,Short,12
6,2016-06-21,3585211803185,4,175898,81,SALE,545,1,41,10,20.0,920,Paris,Centre commercial,Succursale,Paris,116B,NaN,NaN,41
7,2016-06-21,7000000000201,4,175898,81,SALE,545,1,0,0,100.0,920,Paris,Centre commercial,Succursale,Paris,FAVO,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332731,2016-08-31,3585211634772,240296,236270,81,SALE,11,1,37,0,0.0,541,Province,Centre ville,Succursale,Province,105B,Corseterie,Triangle/Brassiere,37
332732,2016-08-31,3585211793813,240296,236270,81,SALE,11,1,49,0,0.0,541,Province,Centre ville,Succursale,Province,245,Homewear,Homewear_Ensemble,49
332733,2016-08-31,3585211635557,240296,236270,81,SALE,11,1,26,0,0.0,541,Province,Centre ville,Succursale,Province,127,Corseterie,Slip,26
332734,2016-08-31,3585211862847,240296,236270,81,SALE,11,1,0,0,100.0,541,Province,Centre ville,Succursale,Province,FAVO,NaN,NaN,0


In [18]:

DATA_TICKET = Matrice_travail_OK.groupby(["ID_INDIVIDU","DATE_ACHAT","CODE_BOUTIQUE","NUM_TICKET"],as_index=False).agg(
    {"PRIX_AP_REMISE" : ["sum", "mean"], "EAN" : "count"})

DATA_TICKET.columns = ["ID_INDIVIDU","DATE_ACHAT", "CODE_BOUTIQUE" ,"NUM_TICKET","CA_VISITE", "PRIX_MOYEN", "NB_PRODUITS"]
#print(DATA_TICKET)

DATA_INDIV = DATA_TICKET.groupby("ID_INDIVIDU",as_index=False).agg(
    {"CA_VISITE" :["count","mean", "sum"], "NB_PRODUITS" : "mean"})

DATA_INDIV.columns = ['_'.join(col).strip() for col in DATA_INDIV.columns.values]
DATA_INDIV.rename(columns = {"ID_INDIVIDU_" : "ID_INDIVIDU","CA_VISITE_count" : "NB_VISITES","CA_VISITE_mean" :"CA_MOY_VISITE","CA_VISITE_sum" : "MONTANT_CUMULE","NB_PRODUITS_mean" : "NB_PRDT_MOY_VISITE" }, inplace=True)


#CALCUL PAR INDIVIDU 

Matrice_travail_OK = Matrice_travail_OK.merge(DF_INDIVIDU[["ID_INDIVIDU","CODE_MAGASIN"]], on='ID_INDIVIDU', how='left')
Matrice_travail_OK["MAGASIN_GESTIO"] = np.where(Matrice_travail_OK["CODE_MAGASIN"] == Matrice_travail_OK["CODE_BOUTIQUE"], 1, 0)



INFO_INDIV = Matrice_travail_OK.groupby("ID_INDIVIDU").agg({
    "DATE_ACHAT" : lambda x :(date_extract - max(x)).days ,"CODE_BOUTIQUE" : 'nunique',"Ligne" : 'nunique',"Famille" : 'nunique',"PRIX_AP_REMISE" : lambda x : len([ i for i in x if i == 0])})

INFO_INDIV.columns = ["RECENCE","NB_MAG_DIFF","NB_LIGNES_DIFF","NB_FAM_DIFF","NB_CADEAUX"]

print(INFO_INDIV)


             RECENCE  NB_MAG_DIFF  NB_LIGNES_DIFF  NB_FAM_DIFF  NB_CADEAUX
ID_INDIVIDU                                                               
4                 39            4               3            4           1
34               498            1               2            3           6
91               143            1               2            8           2
99                95            1               2            6           2
449              562            1               1            2           0
...              ...          ...             ...          ...         ...
240219             0            1               1            3           1
240221             0            1               1            4           1
240269             0            1               1            3           1
240290             0            1               2            3           1
240296             0            1               2            4           1

[16608 rows x 5 columns]

In [19]:
DATA_GEST = DATA_TICKET.merge(DF_INDIVIDU[["ID_INDIVIDU","CODE_MAGASIN"]], on='ID_INDIVIDU', how='left')
DATA_GEST["MAGASIN_GESTIO"] = np.where(DATA_GEST["CODE_MAGASIN"] == DATA_GEST["CODE_BOUTIQUE"], 1, 0)
DATA_GEST = DATA_GEST.groupby("ID_INDIVIDU").agg({"CODE_BOUTIQUE" : "count","MAGASIN_GESTIO" : "sum"})
PART_VIST_MAG_GEST = DATA_GEST["MAGASIN_GESTIO"] / DATA_GEST["CODE_BOUTIQUE"]
PART_VIST_MAG_GEST.describe()

INFO_INDIV["PART_VIST_MAG_GEST"] = PART_VIST_MAG_GEST


df_Matrice_finale = INFO_INDIV.merge(DATA_INDIV, on='ID_INDIVIDU')
df_Matrice_finale = df_Matrice_finale.merge(DATA_TICKET, on='ID_INDIVIDU', how='left')
df_Matrice_finale = df_Matrice_finale.loc[df_Matrice_finale["NB_VISITES"] > 0] 
df_Matrice_finale

,ID_INDIVIDU,RECENCE,NB_MAG_DIFF,NB_LIGNES_DIFF,NB_FAM_DIFF,NB_CADEAUX,PART_VIST_MAG_GEST,NB_VISITES,CA_MOY_VISITE,MONTANT_CUMULE,NB_PRDT_MOY_VISITE,DATE_ACHAT,CODE_BOUTIQUE,NUM_TICKET,CA_VISITE,PRIX_MOYEN,NB_PRODUITS
0,4,39,4,3,4,1,0.0,6,79.166667,475,2.166667,2015-12-14,730,10,69,69.000000,1
1,4,39,4,3,4,1,0.0,6,79.166667,475,2.166667,2016-06-21,920,545,53,17.666667,3
2,4,39,4,3,4,1,0.0,6,79.166667,475,2.166667,2016-07-02,920,513,75,75.000000,1
3,4,39,4,3,4,1,0.0,6,79.166667,475,2.166667,2016-07-06,920,536,65,32.500000,2
4,4,39,4,3,4,1,0.0,6,79.166667,475,2.166667,2016-07-20,753,25,124,31.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57645,240219,0,1,1,3,1,1.0,1,138.000000,138,5.000000,2016-08-31,642,6,138,27.600000,5
57646,240221,0,1,1,4,1,1.0,1,142.000000,142,5.000000,2016-08-31,642,7,142,28.400000,5
57647,240269,0,1,1,3,1,1.0,1,245.000000,245,8.000000,2016-08-31,642,5,245,30.625000,8
57648,240290,0,1,2,3,1,1.0,1,81.000000,81,5.000000,2016-08-31,491,4,81,16.200000,5
